<a href="https://colab.research.google.com/github/MohaMAZ/webscraping/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import pandas as pd
import string
import re

In [ ]:
import re

In [ ]:
my_url = "https://medicament.ma/listing-des-medicaments"

In [ ]:
client = uReq(my_url)
client

In [ ]:
page_html = client.read().decode("utf-8")
page_html
client.close()

In [ ]:
page_soup = soup(page_html)
j = int(int(page_soup.findAll('strong')[0].text)/20)+1
j

30

In [ ]:
my_url = "https://medicament.ma/listing-des-medicaments/?lettre="+'D'
client = uReq(my_url)
page_html = client.read()
client.close()
page_soup = soup(page_html)
details = page_soup.findAll('strong')[0].text
print(details)

332


In [ ]:
my_url = "https://medicament.ma/listing-des-medicaments/?lettre="+'D'
client = uReq(my_url)
page_html = client.read()
client.close()
page_soup = soup(page_html)
j = int((int(page_soup.findAll('strong')[0].text)-0.5)/20)+1
print(j)

17


In [ ]:
for k in ["A","B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U"", "V", "W", "X", "Y", "Z"]:
  df = pd.DataFrame(columns = ['Prefix', 'Name', 'Price'])
  my_url = "https://medicament.ma/listing-des-medicaments/?lettre="+k
  client = uReq(my_url)
  page_html = client.read()
  client.close()
  page_soup = soup(page_html)
  j = int((int(page_soup.findAll('strong')[0].text)-0.5)/20)+1
  for i in range(1,j+1):
    my_url = "https://medicament.ma/listing-des-medicaments/page/"+str(i)+"/?lettre="+k
    client = uReq(my_url)
    page_html = client.read()
    client.close()
    page_soup = soup(page_html)
    details = page_soup.findAll('span',{"class" : "details"})
    for item in details :
      res = re.split("\n",item.text)
      try:
        df = df.append({'Prefix' : k , 'Name' : res[1].strip(), 'Price' : re.split(' ',re.split(": ",res[2])[1])[0]}, 
                    ignore_index = True)
      except:
        df = df.append({'Prefix' : k , 'Name' : item.text, 'Price' : 'Null'}, 
                      ignore_index = True)
  df.to_csv(k+"_data.csv",index=False)